In [16]:
#pip install backtrader yfinance pandas numpy
# pip install pyfolio-reloaded yfinance==0.2.37 pandas==2.2.3
# pip install backtrader==1.9.76.123 yfinance==0.2.18 pandas==1.5.3
pip install backtrader==1.9.76.123

SyntaxError: invalid syntax (1246316869.py, line 4)

In [3]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# 检查Backtrader版本
print(f"Backtrader版本: {bt.__version__}")

  # tickers = ['SPY','QQQ','SHY','GLD','SOYB','VNM','INDA','BTC-USD','159545.SZ'] 
  #       allocation = [0.20, 0.12, 0.20, 0.25, 0.03, 0.03, 0.05, 0.07, 0.05]    

# 1. 定义投资组合
portfolio = {
    'SPY': 0.20,   # 苹果公司，权重40%
    'QQQ': 0.12,   # 微软，权重30%
    'SHY': 0.20,   # 亚马逊，权重20%
    'GLD': 0.25,   # 谷歌，权重10%
    'SOYB': 0.07,
    'VNM': 0.03,
    'INDA': 0.06,
    'BTC-USD': 0.07,
}

# 2. 设置回测参数
start_date = '2020-01-01'
end_date = '2023-12-31'
initial_cash = 100000  # 初始资金10万美元
risk_free_rate = 0.03   # 假设无风险利率为3%

# 3. 数据准备函数（适配旧版Backtrader）
def prepare_data(tickers, start, end):
    datafeeds = []
    for ticker in tickers:
        df = yf.download(ticker, start=start, end=end)
        # 确保列名符合Backtrader 1.9.76要求
        df = df.rename(columns={
            'Open': 'open',
            'High': 'high',
            'Low': 'low',
            'Close': 'close',
            'Adj Close': 'adjclose',
            'Volume': 'volume'
        })
        df = df.dropna()
        # 旧版Backtrader需要这样创建数据源
        datafeed = bt.feeds.PandasData(
            dataname=df,
            datetime=None,  # 禁用自动日期检测
            open=0,
            high=1,
            low=2,
            close=3,
            volume=5,
            openinterest=-1
        )
        datafeeds.append((ticker, datafeed))
    return datafeeds

print("下载并准备数据中...")
datafeeds = prepare_data(portfolio.keys(), start_date, end_date)

# 4. 策略类（适配旧版）
class PortfolioStrategy(bt.Strategy):
    params = dict(
        allocation=list(portfolio.values())  # 旧版需要列表形式
    )
    
    def __init__(self):
        self.portfolio_value = []
        self.dates = []
        self._last_rebalance = None
        self.assets = self.datas  # 旧版兼容性
        
    def next(self):
        self.portfolio_value.append(self.broker.getvalue())
        self.dates.append(self.datetime.date())
        
        if self._last_rebalance is None:
            self._last_rebalance = self.datetime.date()
            return
            
        if self.datetime.date().month != self._last_rebalance.month:
            self.rebalance()
            self._last_rebalance = self.datetime.date()
    
    def rebalance(self):
        total = self.broker.getvalue()
        for i, d in enumerate(self.assets):
            self.order_target_percent(d, target=self.params.allocation[i])
    
    def start(self):
        self._last_rebalance = None
        self.rebalance()

# 5. 回测运行函数
def run_backtest():
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(initial_cash)
    
    # 添加数据（旧版方式）
    for ticker, data in datafeeds:
        cerebro.adddata(data, name=ticker)
    
    # 添加策略
    cerebro.addstrategy(PortfolioStrategy)
    
    # 分析器配置（旧版兼容）
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, 
                      timeframe=bt.TimeFrame.Days,
                      riskfreerate=risk_free_rate,
                      annualize=True,
                      _name='sharpe')
    
    print(f'初始资金: {cerebro.broker.getvalue():.2f}')
    results = cerebro.run()
    print(f'最终资金: {cerebro.broker.getvalue():.2f}')
    return results[0]

# 6. 运行回测
print("运行回测...")
try:
    result = run_backtest()
    
    # 7. 计算指标
    def get_metrics(strat):
        ret = strat.analyzers.returns.get_analysis()
        dd = strat.analyzers.drawdown.get_analysis()
        sharpe = strat.analyzers.sharpe.get_analysis()
        
        # 年化收益
        ann_return = ret['rnorm100'] / 100
        
        # 最大回撤
        max_dd = dd['max']['drawdown'] / 100
        
        # 夏普比率
        sharpe_ratio = sharpe['sharperatio']
        
        # 波动率
        vals = np.array(strat.portfolio_value)
        daily_ret = np.diff(vals) / vals[:-1]
        vol = np.std(daily_ret) * np.sqrt(252)
        
        # Alpha (简化版)
        spy = yf.download('SPY', start_date, end_date)['Close']
        spy_ret = (spy[-1]/spy[0])**(252/len(spy)) - 1
        alpha = ann_return - spy_ret
        
        return {
            'Annual Return': ann_return,
            'Max Drawdown': max_dd,
            'Volatility': vol,
            'Sharpe Ratio': sharpe_ratio,
            'Alpha': alpha
        }
    
    metrics = get_metrics(result)
    
    print("\n投资组合表现:")
    print("="*40)
    print(f"年化收益率: {metrics['Annual Return']:.2%}")
    print(f"最大回撤: {metrics['Max Drawdown']:.2%}")
    print(f"波动率: {metrics['Volatility']:.2%}")
    print(f"夏普比率: {metrics['Sharpe Ratio']:.2f}")
    print(f"阿尔法率: {metrics['Alpha']:.2%}")

except Exception as e:
    print(f"发生错误: {str(e)}")
    print("建议解决方案:")
    print("1. 确保安装了正确版本: pip install backtrader==1.9.76.123")
    print("2. 尝试使用更简单的策略测试")
    print("3. 检查数据下载是否完整")

Backtrader版本: 1.9.78.123
下载并准备数据中...


/tmp/ipykernel_7701/3348941095.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_7701/3348941095.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_7701/3348941095.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_7701/3348941095.py:35: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_7701/3348941095.py:35: FutureWarning: YF.download

运行回测...
初始资金: 100000.00
发生错误: 'tuple' object has no attribute 'lower'
建议解决方案:
1. 确保安装了正确版本: pip install backtrader==1.9.76.123
2. 尝试使用更简单的策略测试
3. 检查数据下载是否完整
